**In this project I have tried to perform Exploratory Data Analysis using SQL integrated with Python. I have used mysql.connector to connect to MySQL database.** 

In [71]:
import pandas as pd

In [2]:
dataset = pd.read_excel("C:/Users/Deepanshu/DA/Datasets/ESD.xlsx")

### Connecting to database

In [72]:
import mysql.connector

In [73]:
conn = mysql.connector.connect(
    host = 'localhost', 
    user = 'root', 
    password = 'root123', 
    database = 'db3'
)

if conn.is_connected():
    print("Connection established! ")

Connection established! 


### Creating a table

In [74]:
cursr = conn.cursor()

In [6]:
query = '''
create table esd (
    EEID varchar(10) not null,
    `Full Name` varchar(255) not null, 
    `Job Title` varchar(255) not null, 
    Department varchar(255) not null, 
    `Business Unit` varchar(255) not null, 
    Gender varchar(10) not null, 
    Ethnicity varchar(50) not null, 
    Age int not null, 
    `Hire Date` date not null, 
    `Annual Salary` decimal(10, 2) not null, 
    `Bonus %` decimal(5, 2) not null, 
    `Country` varchar(100) not null, 
    `City` varchar(100) not null,
    `Exit Date` date
);

'''

cursr.execute(query)

In [7]:
query = '''
show tables;
'''

cursr.execute(query)
result = cursr.fetchall()

In [8]:
# python function to display result in grid format
from tabulate import tabulate

def printResult(result):
    # get the column names
    column_names = [column[0] for column in cursr.description] 
    
    # print the result as table
    print(tabulate(result, headers=column_names, tablefmt='grid'))

In [9]:
printResult(result)

+-----------------+
| Tables_in_db3   |
+=================+
| esd             |
+-----------------+


In [10]:
# python function to display result in python dataframe

def printResultDf(result): 
    # get the column names
    column_names = [column[0] for column in cursr.description]
    
    # print the result as dataframe
    df = pd.DataFrame(result, columns=column_names)
    return df

In [11]:
printResultDf(result)

,Tables_in_db3
0,esd


In [12]:
query = '''
describe esd;
'''

cursr.execute(query)
result = cursr.fetchall()
printResult(result)

+---------------+---------------+--------+-------+-----------+---------+
| Field         | Type          | Null   | Key   | Default   | Extra   |
+===============+===============+========+=======+===========+=========+
| EEID          | varchar(10)   | NO     |       |           |         |
+---------------+---------------+--------+-------+-----------+---------+
| Full Name     | varchar(255)  | NO     |       |           |         |
+---------------+---------------+--------+-------+-----------+---------+
| Job Title     | varchar(255)  | NO     |       |           |         |
+---------------+---------------+--------+-------+-----------+---------+
| Department    | varchar(255)  | NO     |       |           |         |
+---------------+---------------+--------+-------+-----------+---------+
| Business Unit | varchar(255)  | NO     |       |           |         |
+---------------+---------------+--------+-------+-----------+---------+
| Gender        | varchar(10)   | NO     |       | 

### Inserting records in the table

In [13]:
query = '''
insert into esd (EEID, `Full Name`, `Job Title`, Department, `Business Unit`, Gender, Ethnicity, Age, `Hire Date`, `Annual Salary`, `Bonus %`, Country, City, `Exit Date`)
values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

In [14]:
for index, row in dataset.iterrows():
    cursr.execute(query, (row['EEID'], row['Full Name'], row['Job Title'], row['Department'], row['Business Unit'], row['Gender'], row['Ethnicity'], row['Age'], row['Hire Date'], row['Annual Salary'], row['Bonus %'], row['Country'], row['City'], row['Exit Date']))

<hr style="border:1px solid gray">

# Data Preprocessing using SQL

### Checking for duplicate records in the table ESD

In [15]:
# to check for duplicate records in the dataset
query = ''' 
select EEID, count(*)
from esd
group by EEID 
having count(EEID) > 1;
'''

cursr.execute(query)
result = cursr.fetchall()
printResult(result)

+--------+------------+
| EEID   |   count(*) |
+========+============+
| E01639 |          2 |
+--------+------------+
| E04332 |          2 |
+--------+------------+
| E00530 |          2 |
+--------+------------+
| E04732 |          2 |
+--------+------------+
| E00671 |          2 |
+--------+------------+
| E03247 |          2 |
+--------+------------+
| E02185 |          2 |
+--------+------------+
| E03830 |          2 |
+--------+------------+
| E03720 |          2 |
+--------+------------+
| E03824 |          2 |
+--------+------------+
| E00436 |          2 |
+--------+------------+
| E01249 |          2 |
+--------+------------+
| E03349 |          3 |
+--------+------------+
| E01499 |          3 |
+--------+------------+
| E02166 |          2 |
+--------+------------+
| E01525 |          2 |
+--------+------------+
| E00431 |          2 |
+--------+------------+
| E00440 |          2 |
+--------+------------+
| E00972 |          2 |
+--------+------------+
| E04562 |      

In [16]:
# to check if duplicate records exists for a specific person. 
query = '''
select EEID, 'Full Name', Gender, Ethnicity, Age, Country, City, count(*) as duplicate_count
from esd 
group by EEID, 'Full Name', Gender, Ethnicity, Age, Country, City 
having count(*) > 1;
'''

cursr.execute(query)
result = cursr.fetchall()
printResult(result)

+--------+-------------+----------+-------------+-------+-----------+--------+-------------------+
| EEID   | Full Name   | Gender   | Ethnicity   | Age   | Country   | City   | duplicate_count   |
+========+=============+==========+=============+=======+===========+========+===================+
+--------+-------------+----------+-------------+-------+-----------+--------+-------------------+


As we can see, there are duplicate records for EEID but there are no duplicate records for Full Name, Gender, Ethnicity, Age, Country, City. This makes it clear that there are no duplicate records for a person. The duplicate records of EEID do not represent the duplicate records for a specific person. 

To remove the duplicate EEID records, we need to drop the EEID column and create a new column to represent ID column.

**Dropping the old EEID column which contains duplicate EEIDs**

In [17]:
query = '''
alter table esd drop EEID;
'''
cursr.execute(query)

In [18]:
query = '''
describe esd;
'''

cursr.execute(query)
result = cursr.fetchall()
printResult(result)

+---------------+---------------+--------+-------+-----------+---------+
| Field         | Type          | Null   | Key   | Default   | Extra   |
+===============+===============+========+=======+===========+=========+
| Full Name     | varchar(255)  | NO     |       |           |         |
+---------------+---------------+--------+-------+-----------+---------+
| Job Title     | varchar(255)  | NO     |       |           |         |
+---------------+---------------+--------+-------+-----------+---------+
| Department    | varchar(255)  | NO     |       |           |         |
+---------------+---------------+--------+-------+-----------+---------+
| Business Unit | varchar(255)  | NO     |       |           |         |
+---------------+---------------+--------+-------+-----------+---------+
| Gender        | varchar(10)   | NO     |       |           |         |
+---------------+---------------+--------+-------+-----------+---------+
| Ethnicity     | varchar(50)   | NO     |       | 

**Adding a new column as EEID**

In [19]:
# adding a new column ID for reference
query = '''
alter table esd 
add column `ID` int auto_increment primary key;
'''

cursr.execute(query)

In [20]:
query = '''
select * from esd; 
'''

cursr.execute(query)
result = cursr.fetchall()
printResultDf(result)

,Full Name,Job Title,Department,Business Unit,Gender,Ethnicity,Age,Hire Date,Annual Salary,Bonus %,Country,City,Exit Date,ID
0,Emily Davis,Sr. Manger,IT,Research & Development,Female,Black,55,2016-04-08,141604.00,0.15,United States,Seattle,2021-10-16,1
1,Theodore Dinh,Technical Architect,IT,Manufacturing,Male,Asian,59,1997-11-29,99975.00,0.00,China,Chongqing,0001-01-01,2
2,Luna Sanders,Director,Finance,Speciality Products,Female,Caucasian,50,2006-10-26,163099.00,0.20,United States,Chicago,0001-01-01,3
3,Piper Lewis,Field Engineer,Engineering,Research & Development,Female,Caucasian,33,2018-12-22,83990.00,0.00,United States,Chicago,0001-01-01,4
4,Penelope Jordan,Computer Systems Manager,IT,Manufacturing,Female,Caucasian,26,2019-09-27,84913.00,0.07,United States,Chicago,0001-01-01,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Paisley Bryant,Cloud Infrastructure Architect,IT,Manufacturing,Female,Black,37,2016-04-27,91400.00,0.00,United States,Chicago,0001-01-01,996
996,Joshua Ramirez,Vice President,Human Resources,Corporate,Male,Latino,44,2007-09-10,181247.00,0.33,Brazil,Sao Paulo,0001-01-01,997
997,Joshua Martin,Sr. Manger,Human Resources,Research & Development,Male,Black,42,2003-10-20,135558.00,0.14,United States,Phoenix,0001-01-01,998
998,Charles Moore,Analyst,Accounting,Speciality Products,Male,Caucasian,49,2011-12-17,56878.00,0.00,United States,Seattle,0001-01-01,999


In [21]:
# adding a new column for Employee ID
query = '''
alter table esd add column EEID varchar(10); 
'''

cursr.execute(query)

In [22]:
# updating the new column in the desired format. 
query = '''
update esd 
set EEID = concat('E', lpad(ID, 5, '0'));
'''

cursr.execute(query)

In [23]:
# now deleting the ID column which contains serial numbers
query = '''
alter table esd drop column ID;
'''

cursr.execute(query)

In [24]:
# rearranging the columns of the table, setting EEID as the first column
query = '''
alter table esd 
modify column EEID varchar(10) first;
'''

cursr.execute(query)

In [25]:
# setting EEID as Primary key
query = '''
alter table esd 
add primary key (EEID); 
'''

cursr.execute(query)

**Updating the Exit Date column as NULL where the employee has not left the organisation**

In [26]:
query = '''
update esd 
set `Exit Date` = NULL
where `Exit Date` = '0001-01-01';
'''

cursr.execute(query)

query = '''
select * from esd;
'''

cursr.execute(query)
result = cursr.fetchall()
printResultDf(result)

,EEID,Full Name,Job Title,Department,Business Unit,Gender,Ethnicity,Age,Hire Date,Annual Salary,Bonus %,Country,City,Exit Date
0,E00001,Emily Davis,Sr. Manger,IT,Research & Development,Female,Black,55,2016-04-08,141604.00,0.15,United States,Seattle,2021-10-16
1,E00002,Theodore Dinh,Technical Architect,IT,Manufacturing,Male,Asian,59,1997-11-29,99975.00,0.00,China,Chongqing,None
2,E00003,Luna Sanders,Director,Finance,Speciality Products,Female,Caucasian,50,2006-10-26,163099.00,0.20,United States,Chicago,None
3,E00004,Piper Lewis,Field Engineer,Engineering,Research & Development,Female,Caucasian,33,2018-12-22,83990.00,0.00,United States,Chicago,None
4,E00005,Penelope Jordan,Computer Systems Manager,IT,Manufacturing,Female,Caucasian,26,2019-09-27,84913.00,0.07,United States,Chicago,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,E00996,Paisley Bryant,Cloud Infrastructure Architect,IT,Manufacturing,Female,Black,37,2016-04-27,91400.00,0.00,United States,Chicago,None
996,E00997,Joshua Ramirez,Vice President,Human Resources,Corporate,Male,Latino,44,2007-09-10,181247.00,0.33,Brazil,Sao Paulo,None
997,E00998,Joshua Martin,Sr. Manger,Human Resources,Research & Development,Male,Black,42,2003-10-20,135558.00,0.14,United States,Phoenix,None
998,E00999,Charles Moore,Analyst,Accounting,Speciality Products,Male,Caucasian,49,2011-12-17,56878.00,0.00,United States,Seattle,None


### Normalizing tables 
**Creating a new table for location**

This will help to reduce the redundancy of ESD table by replacing the `Country` and `City` columns by `Territory Key` column in ESD table.

In [27]:
# create a view for called territory_lookup
query = '''
create view Territory_Lookup_view as 
select 
    distinct country as Country, city as City
from esd 
order by country, city;
'''

cursr.execute(query)

In [28]:
# creating a table from a view
query = '''
create table Territory_Lookup as 
select * from territory_lookup_view;
'''

cursr.execute(query)

In [29]:
# adding a new column called `Territory Key` in the table territory_lookup
query = '''
alter table territory_lookup 
add column `Territory Key` varchar(5);
'''

cursr.execute(query)

In [30]:
# update the column `Territory Key` with sequential values
# initializing a variable outside outside the update query
query = '''
set @row_number = 0;
'''
cursr.execute(query)

In [31]:
query = '''
update territory_lookup 
set `Territory Key` = concat('t', (@row_number := @row_number + 1))
order by country, city;
'''

cursr.execute(query)
conn.commit()

**replacing the Country and City column from esd with territory key**

In [32]:
# add the `Teritory Key` column in esd table. 
query = '''
alter table esd 
add column `Territory Key` varchar(5);
'''

cursr.execute(query)

In [33]:
# updating `Territory Key` in esd table by joining territory_lookup table.
query = '''
update esd as t1 join territory_lookup as t2 
on t1.country = t2.country and t1.city = t2.city
set t1.`territory key` = t2.`territory key`;
'''
cursr.execute(query)

In [34]:
# removing the columns country and city
query = '''
alter table esd 
drop country, 
drop city;
'''

cursr.execute(query)

In [35]:
# rearranging columns in esd table
query = '''
alter table esd 
modify column `Territory Key` varchar(5) after `Bonus %`;
'''

cursr.execute(query)

In [36]:
query = '''
select * from esd;
'''

cursr.execute(query)
result = cursr.fetchall()
printResultDf(result)

,EEID,Full Name,Job Title,Department,Business Unit,Gender,Ethnicity,Age,Hire Date,Annual Salary,Bonus %,Territory Key,Exit Date
0,E00001,Emily Davis,Sr. Manger,IT,Research & Development,Female,Black,55,2016-04-08,141604.00,0.15,t13,2021-10-16
1,E00002,Theodore Dinh,Technical Architect,IT,Manufacturing,Male,Asian,59,1997-11-29,99975.00,0.00,t6,None
2,E00003,Luna Sanders,Director,Finance,Speciality Products,Female,Caucasian,50,2006-10-26,163099.00,0.20,t9,None
3,E00004,Piper Lewis,Field Engineer,Engineering,Research & Development,Female,Caucasian,33,2018-12-22,83990.00,0.00,t9,None
4,E00005,Penelope Jordan,Computer Systems Manager,IT,Manufacturing,Female,Caucasian,26,2019-09-27,84913.00,0.07,t9,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,E00996,Paisley Bryant,Cloud Infrastructure Architect,IT,Manufacturing,Female,Black,37,2016-04-27,91400.00,0.00,t9,None
996,E00997,Joshua Ramirez,Vice President,Human Resources,Corporate,Male,Latino,44,2007-09-10,181247.00,0.33,t3,None
997,E00998,Joshua Martin,Sr. Manger,Human Resources,Research & Development,Male,Black,42,2003-10-20,135558.00,0.14,t12,None
998,E00999,Charles Moore,Analyst,Accounting,Speciality Products,Male,Caucasian,49,2011-12-17,56878.00,0.00,t13,None


<hr style="border:1px solid gray">

# Exploratory Data Analysis

**Q. How many employees are represented in this dataset?** 

In [37]:
query = '''
select count(EEID) from esd;
'''

cursr.execute(query)
result = cursr.fetchall()
printResult(result)

+---------------+
|   count(EEID) |
+===============+
|          1000 |
+---------------+


**Q. How many active employees are present?**

In [38]:
# creating a view
query = '''
create view Active_Employees_View as 
select * 
from esd 
where `Exit Date` is NULL;
'''

cursr.execute(query)

In [40]:
query = '''
select * from active_employees_view;
'''

cursr.execute(query)
result = cursr.fetchall()
printResultDf(result)

,EEID,Full Name,Job Title,Department,Business Unit,Gender,Ethnicity,Age,Hire Date,Annual Salary,Bonus %,Territory Key,Exit Date
0,E00002,Theodore Dinh,Technical Architect,IT,Manufacturing,Male,Asian,59,1997-11-29,99975.00,0.00,t6,None
1,E00003,Luna Sanders,Director,Finance,Speciality Products,Female,Caucasian,50,2006-10-26,163099.00,0.20,t9,None
2,E00004,Piper Lewis,Field Engineer,Engineering,Research & Development,Female,Caucasian,33,2018-12-22,83990.00,0.00,t9,None
3,E00005,Penelope Jordan,Computer Systems Manager,IT,Manufacturing,Female,Caucasian,26,2019-09-27,84913.00,0.07,t9,None
4,E00006,Elias Figueroa,Sr. Manger,Finance,Corporate,Male,Latino,45,2021-12-24,144754.00,0.15,t12,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
910,E00996,Paisley Bryant,Cloud Infrastructure Architect,IT,Manufacturing,Female,Black,37,2016-04-27,91400.00,0.00,t9,None
911,E00997,Joshua Ramirez,Vice President,Human Resources,Corporate,Male,Latino,44,2007-09-10,181247.00,0.33,t3,None
912,E00998,Joshua Martin,Sr. Manger,Human Resources,Research & Development,Male,Black,42,2003-10-20,135558.00,0.14,t12,None
913,E00999,Charles Moore,Analyst,Accounting,Speciality Products,Male,Caucasian,49,2011-12-17,56878.00,0.00,t13,None


In [41]:
# listing all tables in database
query = '''
show full tables;
'''

cursr.execute(query)
result = cursr.fetchall()
printResult(result)

+-----------------------+--------------+
| Tables_in_db3         | Table_type   |
+=======================+==============+
| active_employees_view | VIEW         |
+-----------------------+--------------+
| esd                   | BASE TABLE   |
+-----------------------+--------------+
| territory_lookup      | BASE TABLE   |
+-----------------------+--------------+
| territory_lookup_view | VIEW         |
+-----------------------+--------------+


**Q. What is the distribution of gender among the employees?**

In [43]:
# using a view
query = '''
select 
    gender as Gender, 
    count(*) as 'Gender Count'
from active_employees_view
group by gender;
'''

cursr.execute(query)
result = cursr.fetchall()
printResult(result)

+----------+----------------+
| Gender   |   Gender Count |
+==========+================+
| Male     |            436 |
+----------+----------------+
| Female   |            479 |
+----------+----------------+


In [44]:
# gender distribution by department
query = '''
select 
    department as Department, 
    gender as Gender, 
    count(*) as 'Gender Count'
from active_employees_view
group by department, gender
order by department;
'''

cursr.execute(query)
result = cursr.fetchall()
printResult(result)

+-----------------+----------+----------------+
| Department      | Gender   |   Gender Count |
+=================+==========+================+
| Accounting      | Female   |             52 |
+-----------------+----------+----------------+
| Accounting      | Male     |             37 |
+-----------------+----------+----------------+
| Engineering     | Female   |             71 |
+-----------------+----------+----------------+
| Engineering     | Male     |             70 |
+-----------------+----------+----------------+
| Finance         | Female   |             65 |
+-----------------+----------+----------------+
| Finance         | Male     |             46 |
+-----------------+----------+----------------+
| Human Resources | Female   |             59 |
+-----------------+----------+----------------+
| Human Resources | Male     |             55 |
+-----------------+----------+----------------+
| IT              | Female   |            112 |
+-----------------+----------+----------

In [45]:
# Pivoting the data to show count of active Female and Male employees
query = '''
select 
    department as Department, 
    sum(case when gender = 'Female' then 1 else 0 end) as Female, 
    sum(case when gender = 'Male' then 1 else 0 end) as Male, 
    count(*) as 'Total Count'
from Active_Employees_View
group by department

union 

select 
    'Total' as Department, 
    sum(case when gender = 'Female' then 1 else 0 end) as Female, 
    sum(case when gender = 'Male' then 1 else 0 end) as Male, 
    count(*) as 'Total Count'
from active_employees_View;
'''

cursr.execute(query)
result = cursr.fetchall()
printResult(result)

+-----------------+----------+--------+---------------+
| Department      |   Female |   Male |   Total Count |
+=================+==========+========+===============+
| IT              |      112 |    113 |           225 |
+-----------------+----------+--------+---------------+
| Finance         |       65 |     46 |           111 |
+-----------------+----------+--------+---------------+
| Engineering     |       71 |     70 |           141 |
+-----------------+----------+--------+---------------+
| Sales           |       70 |     60 |           130 |
+-----------------+----------+--------+---------------+
| Accounting      |       52 |     37 |            89 |
+-----------------+----------+--------+---------------+
| Human Resources |       59 |     55 |           114 |
+-----------------+----------+--------+---------------+
| Marketing       |       50 |     55 |           105 |
+-----------------+----------+--------+---------------+
| Total           |      479 |    436 |         

**Q. What are the most common job titles?**

In [46]:
# using windows function
query = '''
select 
    distinct `job title` as `Job Title`, 
    count(EEID) over (partition by `Job Title`) as `Count of Job Title`
from esd 
order by `Count of Job Title` desc
limit 0, 5;
'''

cursr.execute(query)
result = cursr.fetchall()
printResult(result)

+----------------+----------------------+
| Job Title      |   Count of Job Title |
+================+======================+
| Director       |                  121 |
+----------------+----------------------+
| Sr. Manger     |                  110 |
+----------------+----------------------+
| Vice President |                  105 |
+----------------+----------------------+
| Manager        |                   98 |
+----------------+----------------------+
| Sr. Analyst    |                   70 |
+----------------+----------------------+


In [47]:
query = '''
select 
    `job title` as 'Job Title', 
    count(*) as Count
from esd
group by `Job title`
order by Count desc;
'''

cursr.execute(query)
result = cursr.fetchall()
printResult(result)

+--------------------------------+---------+
| Job Title                      |   Count |
+================================+=========+
| Director                       |     121 |
+--------------------------------+---------+
| Sr. Manger                     |     110 |
+--------------------------------+---------+
| Vice President                 |     105 |
+--------------------------------+---------+
| Manager                        |      98 |
+--------------------------------+---------+
| Sr. Analyst                    |      70 |
+--------------------------------+---------+
| Analyst II                     |      53 |
+--------------------------------+---------+
| Analyst                        |      51 |
+--------------------------------+---------+
| Field Engineer                 |      21 |
+--------------------------------+---------+
| Computer Systems Manager       |      21 |
+--------------------------------+---------+
| Account Representative         |      21 |
+---------

**Q. What is the average age of employees in the company?**

In [49]:
query = '''
select 
    department as Department, 
    avg(age) as 'Average Age'
from active_employees_view
group by department
'''

cursr.execute(query)
result = cursr.fetchall()
printResult(result)

+-----------------+---------------+
| Department      |   Average Age |
+=================+===============+
| IT              |       44.3467 |
+-----------------+---------------+
| Finance         |       45.8919 |
+-----------------+---------------+
| Engineering     |       45.6879 |
+-----------------+---------------+
| Sales           |       43.6846 |
+-----------------+---------------+
| Accounting      |       43.7303 |
+-----------------+---------------+
| Human Resources |       44.3596 |
+-----------------+---------------+
| Marketing       |       43.6952 |
+-----------------+---------------+


**Q. Can you provide a breakdown of employee ages by gender?**

In [50]:
# pivoting the data
query = '''
select 
    department as Department, 
    avg(case when gender = 'Female' then age end) as 'Female avg age', 
    avg(case when gender = 'Male' then age end) as 'Male avg age'
from active_employees_view 
group by department;
'''

cursr.execute(query)
result = cursr.fetchall()
printResult(result)

+-----------------+------------------+----------------+
| Department      |   Female avg age |   Male avg age |
+=================+==================+================+
| IT              |          44.7589 |        43.9381 |
+-----------------+------------------+----------------+
| Finance         |          44.7385 |        47.5217 |
+-----------------+------------------+----------------+
| Engineering     |          43.5634 |        47.8429 |
+-----------------+------------------+----------------+
| Sales           |          43.7143 |        43.65   |
+-----------------+------------------+----------------+
| Accounting      |          43.9423 |        43.4324 |
+-----------------+------------------+----------------+
| Human Resources |          44.7119 |        43.9818 |
+-----------------+------------------+----------------+
| Marketing       |          43.4    |        43.9636 |
+-----------------+------------------+----------------+


**Q. What is the distribution of ethnicities among the employees?**

In [51]:
query = '''
select 
    ethnicity as Ethnicities, 
    count(*) as Count
from esd
group by ethnicity; 
'''

cursr.execute(query)
result = cursr.fetchall()
printResult(result)

+---------------+---------+
| Ethnicities   |   Count |
+===============+=========+
| Black         |      74 |
+---------------+---------+
| Asian         |     404 |
+---------------+---------+
| Caucasian     |     271 |
+---------------+---------+
| Latino        |     251 |
+---------------+---------+


In [57]:
# pivoting the data 
query = '''
select 
    department as Department, 
    sum(case when ethnicity = 'Black' then 1 end) as 'Black Ethnicity', 
    sum(case when ethnicity = 'Caucasian' then 1 end) as 'Caucasian Ethnicity', 
    sum(case when ethnicity = 'Asian' then 1 end) as 'Asian Ethnicity', 
    sum(case when ethnicity = 'Latino' then 1 end) as 'Latino Ethnicity'
from esd 
group by department;
'''

cursr.execute(query)
result = cursr.fetchall()
printResult(result)

+-----------------+-------------------+-----------------------+-------------------+--------------------+
| Department      |   Black Ethnicity |   Caucasian Ethnicity |   Asian Ethnicity |   Latino Ethnicity |
+=================+===================+=======================+===================+====================+
| IT              |                20 |                    75 |                93 |                 53 |
+-----------------+-------------------+-----------------------+-------------------+--------------------+
| Finance         |                 9 |                    27 |                51 |                 33 |
+-----------------+-------------------+-----------------------+-------------------+--------------------+
| Engineering     |                 7 |                    44 |                56 |                 51 |
+-----------------+-------------------+-----------------------+-------------------+--------------------+
| Sales           |                13 |                

**Q. Employee distribution as per country and city**

In [66]:
# joining the tables esd and territory_lookup
query = '''
select 
    t2.country as Country, 
    t2.city as City,
    count(*) as `Employee Count`
from db3.esd as t1 join db3.territory_lookup as t2 on t1.`territory key` = t2.`territory key`
group by Country, City
order by Country, City;
'''

cursr.execute(query)
result = cursr.fetchall()
printResult(result)

+---------------+----------------+------------------+
| Country       | City           |   Employee Count |
+===============+================+==================+
| Brazil        | Manaus         |               42 |
+---------------+----------------+------------------+
| Brazil        | Rio de Janerio |               44 |
+---------------+----------------+------------------+
| Brazil        | Sao Paulo      |               53 |
+---------------+----------------+------------------+
| China         | Beijing        |               55 |
+---------------+----------------+------------------+
| China         | Chengdu        |               46 |
+---------------+----------------+------------------+
| China         | Chongqing      |               65 |
+---------------+----------------+------------------+
| China         | Shanghai       |               52 |
+---------------+----------------+------------------+
| United States | Austin         |               99 |
+---------------+-----------

In [ ]:
query = '''
drop procedure EmployeeCountByCity_SP;
'''

cursr.execute(query)

In [67]:
# creating a stored procedure

query = '''
create procedure EmpByCity_Procedure(in country_name varchar(50))

begin
    select 
        t2.country as Country, 
        t2.city as City,
        Count(*) as `Employee Count`
    from esd as t1 join territory_lookup as t2 on t1.`territory key` = t2.`territory key`
    group by Country, City
    having t2.country = country_name;
end;
'''

try:
    # Executing the SQL command
    cursr.execute(query)
    print("Stored procedure created successfully.")
except mysql.connector.Error as err:
    print(f"Error: {err}")

Stored procedure created successfully.


*note:  DELIMITER command is specific to MySQL command-line tools and is not recognized by mysql.connector in Python. In Python, you don't need to change the delimiter, so you can write your stored procedure without it.*

In [68]:
userInput = input("Enter Country name: ")

Enter Country name: Brazil


In [69]:
query = '''
call EmpByCity_Procedure(%s)
'''

cursr.execute(query, (userInput, ))
result = cursr.fetchall()
printResult(result)

+-----------+----------------+------------------+
| Country   | City           |   Employee Count |
+===========+================+==================+
| Brazil    | Manaus         |               42 |
+-----------+----------------+------------------+
| Brazil    | Rio de Janerio |               44 |
+-----------+----------------+------------------+
| Brazil    | Sao Paulo      |               53 |
+-----------+----------------+------------------+
